# Skywalker algorithm against the two mirror system

In [1]:
SIMULATION = True

goal1 = 210 # Target pixel on first imager. NOTE: Don't do the math for flipped image
goal2 = 270 # Target pixel on second imager. NOTE: Don't do the math for flipped image
first_steps = 10 # Naive first step for the parameters search
tolerances = 2.0 # Tolerance of each target in Pixels
average = 1 # Number of shots to average over
log_level = "INFO"

# PARAMETERS FOR SIMULATION ONLY
centroid_noise = 5.0 # Noise level of centroid measurements
infinite_yag = False # Assume all yags are infinitely large

goals = [goal1, goal2]

In [2]:
############
# Standard #
############
import os
import sys
import logging
import argparse

###############
# Third Party #
###############
from bluesky import RunEngine
from bluesky.tests.utils import MsgCollector
from bluesky.plans import run_wrapper
from pcdsdevices.sim import pim, source, mirror

In [3]:
"""
Parameters
----------
"""
#Configure logger
log_level = getattr(logging, log_level, None)

#Report invalid logging level
if not isinstance(log_level, int):
    raise ValueError("Invalid log level : {}".format(log_level))

#Create basic configuration
logging.basicConfig(level=log_level,
                format='%(asctime)s - %(levelname)s - %(message)s')

from pswalker.examples  import patch_pims
from pswalker.skywalker import skywalker

#Instantiate simulation
if not SIMULATION:
    print("*"*80)
    print("* WARNING: Running on real devices in Production...")
    print("*"*80)
    input("Press any key to continue...")
    system = homs_system()
    m1h = system['m1h']
    m1h2 = system['m1h2']
    m2h = system['m2h']
    m2h2 = system['m2h2']
    xrtm2 = system['xrtm2']
    xrtm22 = system['xrtm22']
    hx2 = system['hx2']
    dg3 = system['dg3']
    mfxdg1 = system['mfxdg1']
    m1 = m1h
    m2 = m2h
    y1 = hx2
    y2 = dg3
else:
    s = source.Undulator('test_undulator')
    m1 = mirror.OffsetMirror('test_m1h', z=90.510, alpha=0.0014)
    m2 = mirror.OffsetMirror('test_m2h', x=0.0317324, z=101.843, alpha=0.0014)
    y1 = pim.PIM('test_p3h', x=0.0317324, z=103.660,
             zero_outside_yag= not infinite_yag)
    y2 = pim.PIM('test_dg3', x=0.0317324, z=375.000,
             zero_outside_yag= not infinite_yag)
    patch_pims([y1, y2], mirrors=[m1, m2], source=s)

    #Add noise
    y1.centroid_noise = centroid_noise
    y2.centroid_noise = centroid_noise


2017-07-20 15:45:30,268 - INFO - Patching 2 pim(s)


In [4]:
#Create Skywalker plan
plan = skywalker([y1, y2], [m1, m2], 'detector_stats2_centroid_x', 'pitch',
             goals, first_steps=first_steps, tolerances=tolerances,
             averages=average, timeout=10)


In [5]:
#Create RunEngine
RE = RunEngine({})
if SIMULATION:
    collector = MsgCollector()
    RE.msg_hook = collector

In [6]:
RE(run_wrapper(plan))

2017-07-20 15:46:02,662 - INFO - Averaged data yielded TST:test_p3h_detector_stats2_centroid_x is at 2992.5109704925367
2017-07-20 15:46:02,664 - INFO - Unable to yield estimate from model Model(linear)
2017-07-20 15:46:02,665 - INFO - Walking motor, error after step #0 -> -2782.5109704925367
2017-07-20 15:46:02,665 - INFO - No model yielded accurate prediction, using naive plan
2017-07-20 15:46:02,777 - INFO - Averaged data yielded TST:test_p3h_detector_stats2_centroid_x is at 2970.1960811994313
2017-07-20 15:46:02,778 - INFO - Walking motor, error after step #1 -> -2760.1960811994313
2017-07-20 15:46:02,779 - INFO - Using model Model(linear) to determine next step.
2017-07-20 15:46:02,780 - INFO - Adjusting motor MIRR:TST:test_m1h to position 1246.9316150448226
2017-07-20 15:46:02,895 - INFO - Averaged data yielded TST:test_p3h_detector_stats2_centroid_x is at 231.1586845932672
2017-07-20 15:46:02,896 - INFO - Walking motor, error after step #2 -> -21.158684593267196
2017-07-20 15:46

2017-07-20 15:46:05,653 - INFO - Averaged data yielded TST:test_dg3_detector_stats2_centroid_x is at 266.927665334671
2017-07-20 15:46:05,654 - INFO - Walking motor, error after step #1 -> 3.0723346653288672
2017-07-20 15:46:05,655 - INFO - Using model Model(linear) to determine next step.
2017-07-20 15:46:05,656 - INFO - Adjusting motor MIRR:TST:test_m2h to position 1454.9231044430674
2017-07-20 15:46:05,796 - INFO - Averaged data yielded TST:test_dg3_detector_stats2_centroid_x is at 270.8546504498207
2017-07-20 15:46:05,797 - INFO - Succesfully walked to value 270.8546504498207 (target=269.9999999999999) after 2 steps.
2017-07-20 15:46:05,798 - INFO - Found equation of (45.93362325190344, -66559.46130659437) between linear fit of MIRR:TST:test_m2h to TST:test_dg3
2017-07-20 15:46:06,081 - INFO - Averaged data yielded TST:test_p3h_detector_stats2_centroid_x is at 217.5075631044156
2017-07-20 15:46:06,082 - INFO - Unable to yield estimate from model Model(linear)
2017-07-20 15:46:06,08

['8d48437c-766e-44b7-8f09-fa63915f3e9b']

In [7]:
if SIMULATION:
    #Analyze Performance
    print(len(RE.msg_hook.msgs))

507
